<a href="https://colab.research.google.com/github/mikethai/server-tools/blob/master/AudioToText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🗣️ [**AudioToText**](https://github.com/Carleslc/AudioToText)

[![Donate](https://www.ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/carleslc)

### 🛠 [Whisper by OpenAI](https://github.com/openai/whisper)


## [Step 1] ⚙️ Install the required libraries

Click ▶️ button below to install the dependencies for this notebook.

In [3]:
#@title { display-mode: "form" }
import subprocess

from sys import platform as sys_platform

status, ffmpeg_version = subprocess.getstatusoutput("ffmpeg -version")

if status != 0:
  from platform import platform

  if sys_platform == 'linux' and 'ubuntu' in platform().lower():
    !apt install ffmpeg
  else:
    print("Install ffmpeg: https://ffmpeg.org/download.html")
else:
  print(ffmpeg_version.split('\n')[0])

  NO_ROOT_WARNING = '|& grep -v \"WARNING: Running pip as the \'root\' user"' # running in Colab

  !pip install --no-warn-script-location --user --upgrade pip {NO_ROOT_WARNING}
  !pip install --root-user-action=ignore git+https://github.com/openai/whisper.git@v20231117 openai==1.9.0 "numpy<2" scipy deepl pydub cohere ffmpeg-python torch==2.1.0 tensorflow-probability==0.23.0 typing-extensions==4.9.0

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  Cloning https://github.com/openai/whisper.git (to revision v20231117) to /tmp/pip-req-build-gvng_w_1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-gvng_w_1
  Running command git checkout -q e58f28804528831904c3b6f2c0e473f346223433
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 24.8 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depen

## [Step 2] 📁 Upload your audio files to the Files folder

⬅️ Files folder in Google Colab is on the left menu

Almost any audio or video file format is [supported](https://gist.github.com/Carleslc/1d6b922c8bf4a7e9627a6970d178b3a6).

## [Step 2.5] 🎙 Record your own audio ⏺

This is an **optional** step to record your microphone, useful if you do not have an audio file to upload and want to create one.

Run this cell to start recording your microphone.
A button will appear to stop the recording when you're done.

The recording will be saved as `recording.wav` which you can use in the next step `audio_file`.

In [ ]:
# This cell code is a slightly modified version of DotCSV code in the following Colab along with other references:
# https://colab.research.google.com/drive/1CvvYPAFemIZdSOt9fhN541esSlZR7Ic6?usp=sharing

try:
  import io
  import ffmpeg
  import numpy as np

  # Only available in Google Colab
  from google.colab.output import eval_js

  from IPython.display import HTML, Audio
  from scipy.io.wavfile import write, read as wav_read
  from base64 import b64decode
  from os.path import isfile

  AUDIO_HTML = """
  <script>
  var my_div = document.createElement("DIV");
  var my_p = document.createElement("P");
  var my_btn = document.createElement("BUTTON");
  var t = document.createTextNode("Starting recording...");

  my_btn.appendChild(t);
  my_div.appendChild(my_btn);
  document.body.appendChild(my_div);

  var base64data = 0;
  var reader;
  var recorder, gumStream;
  var recordButton = my_btn;

  var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
      bitsPerSecond: 16000,
      mimeType : 'audio/webm;codecs=opus' //codecs=pcm
    };
    recorder = new MediaRecorder(stream, options);
    //recorder = new MediaRecorder(stream);

    recorder.ondataavailable = function(e) {
      var url = URL.createObjectURL(e.data);
      var preview = document.createElement('audio');
      preview.controls = true;
      preview.src = url;
      document.body.appendChild(preview);

      reader = new FileReader();
      reader.readAsDataURL(e.data);
      reader.onloadend = function() {
        base64data = reader.result;
        //console.log("reader.onloadend: " + base64data);
      }
    };
    recorder.start();
    recordButton.innerText = "🔴 Recording... press to STOP";
  };

  navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

  function toggleRecording() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... please wait!";
    }
  }

  // https://stackoverflow.com/a/951057
  function sleep(ms) {
    return new Promise(resolve => setTimeout(resolve, ms));
  }

  var data = new Promise(resolve => {
    recordButton.onclick = () => {
      toggleRecording();

      sleep(2000).then(() => {
        // wait 2000ms for the data to be available...
        //console.log("resolve data: " + base64data);
        resolve(base64data.toString());
      });
    }
  });

  function doneRecording(recording_file) {
    my_div.removeChild(recordButton);
    my_p.innerText = recording_file;
    my_div.appendChild(my_p);
  }

  </script>
  """

  def get_audio():
    display(HTML(AUDIO_HTML))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])

    process = (ffmpeg
      .input('pipe:0')
      .output('pipe:1', format='wav')
      .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
    )
    output, err = process.communicate(input=binary)

    riff_chunk_size = len(output) - 8
    # Break up the chunk size into four bytes, held in b.
    q = riff_chunk_size
    b = []
    for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

    # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
    riff = output[:4] + bytes(b) + output[8:]

    sr, audio = wav_read(io.BytesIO(riff))

    return audio, sr

  recording_file = "recording.wav" #@param {type:"string"}

  if isfile(recording_file):
    print(f"{recording_file} already exists, if you want to create another recording with the same name, delete it first")
  else:
    # record microphone
    audio, sr = get_audio()

    # write recording
    write(recording_file, sr, audio)

    eval_js(f'doneRecording("{recording_file}")')
except ImportError:
  print("Recording only available in Google Colab")

## [Step 3] 👂 Transcribe or Translate

3.1. Choose a `task`:
  - `Transcribe` speech to text in the same language of the source audio file.
  - `Translate to English` speech to text in English.
  
Translation to other languages is not supported with _Whisper_ by default.
You may try to choose the _Transcribe_ task and set your desired `language`, but translation is not guaranteed. However, you can use **_DeepL_** later in the Step 5 to translate the transcription to another language.

3.2. Edit the `audio_file` to match your uploaded file name to transcribe.

- If you want to transcribe multiple files with the same parameters you must separate their file names with commas `,`

3.3. Run this cell and wait for the transcription to complete.

  You can try other parameters if the result with default parameters does not suit your needs.

  [Available models and languages](https://github.com/openai/whisper#available-models-and-languages)

  Setting the `language` to the language of source audio file may provide better results than Auto-Detect.

  You can add an optional initial `prompt` to provide context about the audio or encourage a specific writing style, see the [prompting guide](https://platform.openai.com/docs/guides/speech-to-text/prompting).

  If the execution takes too long to complete you can choose a smaller model in `use_model`, with an accuracy tradeoff, or use the OpenAI API.

  By default the open-source models are used, but you can also use the OpenAI API if the `api_key` parameter is set with your [OpenAI API Key](https://platform.openai.com/account/api-keys), which can improve the inference speed substantially, but it has an associated cost, see [API pricing](https://openai.com/pricing#audio-models).

  When using API some options are fixed: **use_model** is ignored (uses _large-v2_) and **coherence_preference** is ignored (uses _More coherence_).
  
  More parameters are available in the code `options` object.

In [1]:
import os, subprocess

import whisper
from whisper.utils import format_timestamp, get_writer, WriteTXT

import numpy as np

try:
  import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
  pass

import torch

import math

from openai import OpenAI

# select task

task = "Transcribe" #@param ["Transcribe", "Translate to English"]

task = "transcribe" if task == "Transcribe" else "translate"

# select audio file

audio_file = "/content/12-10談自結.m4a" #@param {type:"string"}

audio_files = list(map(lambda audio_path: audio_path.strip(), audio_file.split(',')))

for audio_path in audio_files:
  if not os.path.isfile(audio_path):
    raise FileNotFoundError(audio_path)

# set model

use_model = "medium" #@param ["tiny", "base", "small", "medium", "large-v1", "large-v2"]

# select language

language = "Chinese" #@param ["Auto-Detect", "Afrikaans", "Albanian", "Amharic", "Arabic", "Armenian", "Assamese", "Azerbaijani", "Bashkir", "Basque", "Belarusian", "Bengali", "Bosnian", "Breton", "Bulgarian", "Burmese", "Castilian", "Catalan", "Chinese", "Croatian", "Czech", "Danish", "Dutch", "English", "Estonian", "Faroese", "Finnish", "Flemish", "French", "Galician", "Georgian", "German", "Greek", "Gujarati", "Haitian", "Haitian Creole", "Hausa", "Hawaiian", "Hebrew", "Hindi", "Hungarian", "Icelandic", "Indonesian", "Italian", "Japanese", "Javanese", "Kannada", "Kazakh", "Khmer", "Korean", "Lao", "Latin", "Latvian", "Letzeburgesch", "Lingala", "Lithuanian", "Luxembourgish", "Macedonian", "Malagasy", "Malay", "Malayalam", "Maltese", "Maori", "Marathi", "Moldavian", "Moldovan", "Mongolian", "Myanmar", "Nepali", "Norwegian", "Nynorsk", "Occitan", "Panjabi", "Pashto", "Persian", "Polish", "Portuguese", "Punjabi", "Pushto", "Romanian", "Russian", "Sanskrit", "Serbian", "Shona", "Sindhi", "Sinhala", "Sinhalese", "Slovak", "Slovenian", "Somali", "Spanish", "Sundanese", "Swahili", "Swedish", "Tagalog", "Tajik", "Tamil", "Tatar", "Telugu", "Thai", "Tibetan", "Turkish", "Turkmen", "Ukrainian", "Urdu", "Uzbek", "Valencian", "Vietnamese", "Welsh", "Yiddish", "Yoruba"]

# other parameters

prompt = "" #@param {type:"string"}

coherence_preference = "More coherence, but may repeat text" #@param ["More coherence, but may repeat text", "Less repetitions, but may have less coherence"]

api_key = '' #@param {type:"string"}

# detect device

if api_key:
  print("Using API")

  from pydub import AudioSegment
  from pydub.silence import split_on_silence
else:
  DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

  print(f"Using {'GPU' if DEVICE == 'cuda' else 'CPU ⚠️'}")

  # https://medium.com/analytics-vidhya/the-google-colab-system-specification-check-69d159597417
  if DEVICE == "cuda":
    !nvidia-smi -L
  else:
    if sys_platform == 'linux':
      !lscpu | grep "Model name" | awk '{$1=$1};1'

    print("Not using GPU can result in a very slow execution")
    print("Ensure Hardware accelerator by GPU is enabled in Google Colab: Runtime > Change runtime type")

    if use_model not in ['tiny', 'base', 'small']:
      print("You may also want to try a smaller model (tiny, base, small)")

# display language

WHISPER_LANGUAGES = [k.title() for k in whisper.tokenizer.TO_LANGUAGE_CODE.keys()]

if language == "Auto-Detect":
  language = "detect"

if language and language != "detect" and language not in WHISPER_LANGUAGES:
  print(f"\nLanguage '{language}' is invalid")
  language = "detect"

if language and language != "detect":
  print(f"\nLanguage: {language}")

# load model

if api_key:
  print()
else:
  MODELS_WITH_ENGLISH_VERSION = ["tiny", "base", "small", "medium"]

  if language == "English" and use_model in MODELS_WITH_ENGLISH_VERSION:
    use_model += ".en"

  print(f"\nLoading {use_model} model... {os.path.expanduser(f'~/.cache/whisper/{use_model}.pt')}")

  model = whisper.load_model(use_model, device=DEVICE)

  print(
      f"Model {use_model} is {'multilingual' if model.is_multilingual else 'English-only'} "
      f"and has {sum(np.prod(p.shape) for p in model.parameters()):,d} parameters.\n"
  )

# set options

## https://github.com/openai/whisper/blob/v20231117/whisper/transcribe.py#L37
## https://github.com/openai/whisper/blob/v20231117/whisper/decoding.py#L81
options = {
    'task': task,
    'verbose': True,
    'fp16': True,
    'best_of': 5,
    'beam_size': 5,
    'patience': None,
    'length_penalty': None,
    'suppress_tokens': '-1',
    'temperature': (0.0, 0.2, 0.4, 0.6, 0.8, 1.0), # float or tuple
    'condition_on_previous_text': coherence_preference == "More coherence, but may repeat text",
    'initial_prompt': prompt or None,
    'word_timestamps': False,
}

if api_key:
  api_client = OpenAI(api_key=api_key)

  api_supported_formats = ['mp3', 'mp4', 'mpeg', 'mpga', 'm4a', 'wav', 'webm']
  api_max_bytes = 25 * 1024 * 1024 # 25 MB

  api_transcribe = api_client.audio.transcriptions if task == 'transcribe' else api_client.audio.translations
  api_transcribe = api_transcribe.create

  api_model = 'whisper-1' # large-v2

  # https://platform.openai.com/docs/api-reference/audio?lang=python
  api_options = {
    'response_format': 'verbose_json',
  }

  if prompt:
    api_options['prompt'] = prompt

  api_temperature = options['temperature'][0] if isinstance(options['temperature'], (tuple, list)) else options['temperature']

  if isinstance(api_temperature, (float, int)):
    api_options['temperature'] = api_temperature
  else:
    raise ValueError("Invalid temperature type, it must be a float or a tuple of floats")
elif DEVICE == 'cpu':
  options['fp16'] = False
  torch.set_num_threads(os.cpu_count())

# execute task
# !whisper "{audio_file}" --task {task} --model {use_model} --output_dir {output_dir} --device {DEVICE} --verbose {options['verbose']}

if task == "translate":
  print("-- TRANSLATE TO ENGLISH --")
else:
  print("-- TRANSCRIPTION --")

results = {} # audio_path to result

for audio_path in audio_files:
  print(f"\nProcessing: {audio_path}\n")

  # detect language
  detect_language = not language or language == "detect"

  if not detect_language:
    options['language'] = language
    source_language_code = whisper.tokenizer.TO_LANGUAGE_CODE.get(language.lower())
  elif not api_key:
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio_path)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)

    source_language_code = max(probs, key=probs.get)
    options['language'] = whisper.tokenizer.LANGUAGES[source_language_code].title()

    print(f"Detected language: {options['language']}\n")

  # transcribe
  if api_key:
    # API
    if task == "transcribe" and not detect_language:
      api_options['language'] = source_language_code

    source_audio_name_path, source_audio_ext = os.path.splitext(audio_path)
    source_audio_ext = source_audio_ext[1:]

    if source_audio_ext in api_supported_formats:
      api_audio_path = audio_path
      api_audio_ext = source_audio_ext
    else:
      ## convert audio file to a supported format
      if options['verbose']:
        print(f"API supported formats: {','.join(api_supported_formats)}")
        print(f"Converting {source_audio_ext} audio to a supported format...")

      api_audio_ext = 'mp3'

      api_audio_path = f'{source_audio_name_path}.{api_audio_ext}'

      subprocess.run(['ffmpeg', '-i', audio_path, api_audio_path], check=True, capture_output=True)

      if options['verbose']:
        print(api_audio_path, end='\n\n')

    ## split audio file in chunks
    api_audio_chunks = []

    audio_bytes = os.path.getsize(api_audio_path)

    if audio_bytes >= api_max_bytes:
      if options['verbose']:
        print(f"Audio exceeds API maximum allowed file size.\nSplitting audio in chunks...")

      audio_segment_file = AudioSegment.from_file(api_audio_path, api_audio_ext)

      min_chunks = math.ceil(audio_bytes / (api_max_bytes / 2))

      # print(f"Min chunks: {min_chunks}")

      max_chunk_milliseconds = int(len(audio_segment_file) // min_chunks)

      # print(f"Max chunk milliseconds: {max_chunk_milliseconds}")

      def add_chunk(api_audio_chunk):
        api_audio_chunk_path = f"{source_audio_name_path}_{len(api_audio_chunks) + 1}.{api_audio_ext}"
        api_audio_chunk.export(api_audio_chunk_path, format=api_audio_ext)
        api_audio_chunks.append(api_audio_chunk_path)

      def raw_split(big_chunk):
        subchunks = math.ceil(len(big_chunk) / max_chunk_milliseconds)

        for subchunk_i in range(subchunks):
          chunk_start = max_chunk_milliseconds * subchunk_i
          chunk_end = min(max_chunk_milliseconds * (subchunk_i + 1), len(big_chunk))
          add_chunk(big_chunk[chunk_start:chunk_end])

      non_silent_chunks = split_on_silence(audio_segment_file,
                                           seek_step=5, # ms
                                           min_silence_len=1250, # ms
                                           silence_thresh=-25, # dB
                                           keep_silence=True) # needed to aggregate timestamps

      # print(f"Non silent chunks: {len(non_silent_chunks)}")

      current_chunk = non_silent_chunks[0] if non_silent_chunks else audio_segment_file

      for next_chunk in non_silent_chunks[1:]:
        if len(current_chunk) > max_chunk_milliseconds:
          raw_split(current_chunk)
          current_chunk = next_chunk
        elif len(current_chunk) + len(next_chunk) <= max_chunk_milliseconds:
          current_chunk += next_chunk
        else:
          add_chunk(current_chunk)
          current_chunk = next_chunk

      if len(current_chunk) > max_chunk_milliseconds:
        raw_split(current_chunk)
      else:
        add_chunk(current_chunk)

      if options['verbose']:
        print(f'Total chunks: {len(api_audio_chunks)}\n')
    else:
      api_audio_chunks.append(api_audio_path)

    ## process chunks
    result = None

    for api_audio_chunk_path in api_audio_chunks:
      ## API request
      with open(api_audio_chunk_path, 'rb') as api_audio_file:
        api_result = api_transcribe(model=api_model, file=api_audio_file, **api_options)
        api_result = api_result.model_dump() # to dict

      api_segments = api_result['segments']

      if result:
        ## update timestamps
        last_segment_timestamp = result['segments'][-1]['end'] if result['segments'] else 0

        for segment in api_segments:
          segment['start'] += last_segment_timestamp
          segment['end'] += last_segment_timestamp

        ## append new segments
        result['segments'].extend(api_segments)

        if 'duration' in result:
          result['duration'] += api_result.get('duration', 0)
      else:
        ## first request
        result = api_result

        if detect_language:
          print(f"Detected language: {result['language'].title()}\n")

      ## display segments
      if options['verbose']:
        for segment in api_segments:
          print(f"[{format_timestamp(segment['start'])} --> {format_timestamp(segment['end'])}] {segment['text']}")
  else:
    # Open-Source
    result = whisper.transcribe(model, audio_path, **options)

  # fix results formatting
  for segment in result['segments']:
    segment['text'] = segment['text'].strip()

  result['text'] = '\n'.join(map(lambda segment: segment['text'], result['segments']))

  # set results for this audio file
  results[audio_path] = result

Using GPU
GPU 0: Tesla T4 (UUID: GPU-2838dafe-127f-f1b5-e7a7-f1755681906c)

Language: Chinese

Loading medium model... /root/.cache/whisper/medium.pt
Model medium is multilingual and has 762,321,920 parameters.

-- TRANSCRIPTION --

Processing: /content/12-10談自結.m4a

[00:00.000 --> 00:02.000] 大家晚上好
[00:02.000 --> 00:04.000] 那
[00:04.000 --> 00:06.000] 這個今天
[00:06.000 --> 00:08.000] 我們要來講一個叫做
[00:08.000 --> 00:10.000] 台鼓
[00:10.000 --> 00:12.000] 獨有的吧
[00:12.000 --> 00:14.000] 我不知道其他各國會不會有
[00:14.000 --> 00:16.000] 但是我覺得我們台鼓這個東西
[00:16.000 --> 00:18.000] 實在是蠻可愛的
[00:18.000 --> 00:20.000] 那就是什麼,就是字節
[00:20.000 --> 00:22.000] 什麼叫字節
[00:22.000 --> 00:24.000] 就是字節財報
[00:24.000 --> 00:26.000] 字節報表
[00:26.000 --> 00:28.000] 那字節報表會有一個特色
[00:28.000 --> 00:30.000] 呃
[00:30.000 --> 00:32.000] 他們這個通常都是
[00:32.000 --> 00:34.000] 比如說你
[00:34.000 --> 00:36.000] 波動很大,比如說你
[00:36.000 --> 00:38.000] 漲耗金跟停板
[00:38.000 --> 00:40.000] 那今天是11月25
[00:40.000 --> 00:42.000] 像
[00:42.000 --> 00:44.000] 我們前一個音檔講

## [Step 4] 💾 **Save results**

Run this cell to write the transcription as a file output.

Results will be available in the **audio_transcription** folder in the formats selected in `output_formats`.

If you don't see that folder, you may need to refresh 🔄 the Files folder.

Available formats: `txt,vtt,srt,tsv,json`

In [2]:
# set output folder
output_dir = "audio_transcription"

# set output formats: https://github.com/openai/whisper/blob/v20231117/whisper/utils.py#L283
output_formats = "txt" #@param ["txt,vtt,srt,tsv,json", "txt,vtt,srt", "txt,vtt", "txt,srt", "txt", "vtt", "srt", "tsv", "json"] {allow-input: true}
output_formats = output_formats.split(',')

from typing import TextIO

class WriteText(WriteTXT):

  def write_result(self, result: dict, file: TextIO, **kwargs):
    print(result['text'], file=file, flush=True)

def write_result(result, output_format, output_file_name):
  output_format = output_format.strip()

  # start captions in non-zero timestamp (some media players does not detect the first caption)
  fix_vtt = output_format == 'vtt' and result['segments'] and result['segments'][0].get('start') == 0

  if fix_vtt:
    result['segments'][0]['start'] += 1/1000 # +1ms

  # write result in the desired format
  writer = WriteText(output_dir) if output_format == 'txt' else get_writer(output_format, output_dir)
  writer(result, output_file_name)

  if fix_vtt:
    result['segments'][0]['start'] = 0 # reset change

  output_file_path = os.path.join(output_dir, f"{output_file_name}.{output_format}")
  print(output_file_path)

# save results

print("Writing results...")

os.makedirs(output_dir, exist_ok=True)

for audio_path, result in results.items():
  print(end='\n')

  output_file_name = os.path.splitext(os.path.basename(audio_path))[0]

  for output_format in output_formats:
    write_result(result, output_format, output_file_name)

Writing results...

audio_transcription/12-10談自結.txt


## [Step 5] 💬 Translate results with DeepL (API key needed)

This is an **optional** step to translate the transcription to another language using the **DeepL** API.

[Get a DeepL Developer Account API Key](https://www.deepl.com/pro-api)

Set the `deepl_api_key` to translate the transcription to a supported language in `deepl_target_language`.

In [ ]:
import deepl

# translation service options (DeepL Developer Account)

deepl_api_key = "" #@param {type:"string"}

deepl_target_language = "" #@param ["", "Bulgarian", "Chinese (simplified)", "Czech", "Danish", "Dutch", "English (American)", "English (British)", "Estonian", "Finnish", "French", "German", "Greek", "Hungarian", "Indonesian", "Italian", "Japanese", "Korean", "Latvian", "Lithuanian", "Norwegian", "Polish", "Portuguese (Brazilian)", "Portuguese (European)", "Romanian", "Russian", "Slovak", "Slovenian", "Spanish", "Swedish", "Turkish", "Ukrainian"]

deepl_formality = "default" #@param ["default", "formal", "informal"]

deepl_coherence_preference = "Share context between lines" #@param ["Share context between lines", "Translate each line independently"]
deepl_coherence_preference = deepl_coherence_preference == "Share context between lines"

if not deepl_api_key:
  print("Required: deepl_api_key")
  print("Get a DeepL Developer Account API Key: https://www.deepl.com/pro-api")

if not deepl_target_language:
  print("Required: deepl_target_language")
elif deepl_target_language == 'English':
  deepl_target_language = "English (British)"
elif deepl_target_language == 'Chinese':
  deepl_target_language = "Chinese (simplified)"
elif deepl_target_language == 'Portuguese':
  deepl_target_language = "Portuguese (European)"

use_deepl_translation = deepl_api_key and deepl_target_language

if use_deepl_translation:
  if deepl_formality != 'default':
    deepl_formality = 'prefer_more' if deepl_formality == 'formal' else 'prefer_less'

  translated_results = {} # audio_path to translated results

  try:
    deepl_translator = deepl.Translator(deepl_api_key)

    deepl_source_languages = [lang.code.upper() for lang in deepl_translator.get_source_languages()]

    deepl_target_languages_dict = deepl_translator.get_target_languages()
    deepl_target_languages = [lang.name for lang in deepl_target_languages_dict]

    deepl_target_language_code = next(lang.code for lang in deepl_target_languages_dict if lang.name == deepl_target_language).upper()
    target_language_code = deepl_target_language_code.split('-')[0]

    for audio_path, result in results.items():
      deepl_usage = deepl_translator.get_usage()

      if deepl_usage.any_limit_reached:
        print(audio_path)
        raise deepl.DeepLException("Quota for this billing period has been exceeded, message: Quota Exceeded")
      else:
        print(audio_path + '\n')

      # translate results (DeepL)
      source_language_code = whisper.tokenizer.TO_LANGUAGE_CODE.get(result['language'].lower()).upper()

      if (task == 'translate' and target_language_code != 'EN') or (task == 'transcribe' and source_language_code in deepl_source_languages and source_language_code != target_language_code):
        source_lang = source_language_code if task == 'transcribe' else None
        translate_from = f"from {result['language'].title()} [{source_language_code}] " if source_lang else ''
        print(f"DeepL: Translate results {translate_from}to {deepl_target_language} [{deepl_target_language_code}]\n")

        segments = result['segments']

        translated_results[audio_path] = { 'text': '', 'segments': [], 'language': deepl_target_language }

        # segments / request (max 128 KiB / request, so deepl_batch_requests_size is limited to around 1000)
        deepl_batch_requests_size = 200 # 200 segments * ~100 bytes / segment = ~20 KB / request  (~15 minutes of speech)

        for batch_segments in [segments[i:i + deepl_batch_requests_size] for i in range(0, len(segments), deepl_batch_requests_size)]:
          batch_segments_text = [segment['text'] for segment in batch_segments]

          if deepl_coherence_preference:
            batch_segments_text = '<br/>'.join(batch_segments_text)

          # DeepL request
          deepl_results = deepl_translator.translate_text(
              text=batch_segments_text,
              source_lang=source_lang,
              target_lang=deepl_target_language_code,
              formality=deepl_formality,
              split_sentences='nonewlines',
              tag_handling='xml' if deepl_coherence_preference else None,
              ignore_tags='br' if deepl_coherence_preference else None, # used to synchronize sentences with whisper lines but without splitting sentences in DeepL
              outline_detection=False if deepl_coherence_preference else None
          )

          deepl_results_segments = deepl_results.text.split('<br/>') if deepl_coherence_preference else [deepl_result_segment.text for deepl_result_segment in deepl_results]

          for j, translated_text in enumerate(deepl_results_segments):
            segment = batch_segments[j]

            # fix sentence formatting
            translated_text = translated_text.lstrip(',.。 ').rstrip()

            if not deepl_coherence_preference and translated_text and translated_text[-1] in '.。' and segment['text'][-1] not in '.。':
              translated_text = translated_text[:-1]

            # add translated segments
            translated_results[audio_path]['segments'].append(dict(id=segment['id'], start=segment['start'], end=segment['end'], text=translated_text))

            if options['verbose']:
              print(f"[{format_timestamp(segment['start'])} --> {format_timestamp(segment['end'])}] {translated_text}")

        deepl_usage = deepl_translator.get_usage()

        if deepl_usage.character.valid:
          print(f"\nDeepL: Character usage: {deepl_usage.character.count} / {deepl_usage.character.limit} ({100*(deepl_usage.character.count/deepl_usage.character.limit):.2f}%)\n")
      elif source_language_code == target_language_code:
        print(f"Nothing to translate. Results are already in {result['language']}.")
      elif task == 'transcribe' and source_language_code not in deepl_source_languages:
        print(f"DeepL: {result['language']} is not yet supported")
  except deepl.DeepLException as e:
    if isinstance(e, deepl.AuthorizationException) and str(e) == "Authorization failure, check auth_key":
      e = "Authorization failure, check deepl_api_key"
    print(f"\nDeepL: [Error] {e}\n")

  # save translated results (if any)

  if translated_results:
    print("Writing translated results...")

    for audio_path, translated_result in translated_results.items():
      print(end='\n')

      translated_result['text'] = '\n'.join(map(lambda translated_segment: translated_segment['text'], translated_result['segments']))

      output_file_name = os.path.splitext(os.path.basename(audio_path))[0]
      translated_output_file_name = f"{output_file_name}_{deepl_target_language}"

      for output_format in output_formats:
        write_result(translated_result, output_format, translated_output_file_name)